Motivation: Improve ResNet with better representational power.

Introduced cardinality (parallel transformations with grouped convs).

In [ ]:
# # Let's Go Deeper!

# import torch
# from torch import nn
# import torch.nn.functional as F

# class BottlenectBlock(nn.Module):
#     expansion = 4

#     def __init__(self, in_planes, out_channels, stride=1, downsample=None, base_width=4, cardinality=32):
#         super(BottlenectBlock, self).__init__()
#         width = int(out_channels * (base_width / 64.)) * cardinality
#         self.conv1 = nn.Conv2d(in_planes, width, kernel_size=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(width)
    
#         self.relu = nn.ReLU(inplace=True)
#         self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
#         self.bn2 = nn.BatchNorm2d(width) 
        
#         self.conv3 = nn.Conv2d(width, out_channels*self.expansion, kernel_size=1, bias=False)
#         self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        
#         self.downsample = None
#         if stride != 1 or in_planes != out_channels * self.expansion:
#             self.downsample = nn.Sequential(
#                 nn.Conv2d(in_planes, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
#                 nn.BatchNorm2d(out_channels * self.expansion)
#             )
        

#     def forward(self, x):
#         residual = x

#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)

#         out = self.conv2(out)
#         out = self.bn2(out)
#         out = self.relu(out)
        
#         out = self.conv3(out)
#         out = self.bn3(out)

#         if self.downsample is not None:
#             residual = self.downsample(x)

#         out += residual
#         out = self.relu(out)

#         return out


# class ResNet(nn.Module):

#     def __init__(self, block, layers, num_classes, grayscale, cardinality=32, base_width=4):
#         self.inplanes = 64
#         self.cardinality = cardinality
#         self.base_width = base_width
        
#         if grayscale:
#             in_dim = 1
#         else:
#             in_dim = 3
#         super(ResNet, self).__init__()
#         self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
#                                bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.layer1 = self._make_layer(block, 64, layers[0], stride=1, cardinality=self.cardinality, base_width=self.base_width)
#         self.layer2 = self._make_layer(block, 128, layers[1], stride=2, cardinality=self.cardinality, base_width=self.base_width)
#         self.layer3 = self._make_layer(block, 256, layers[2], stride=2, cardinality=self.cardinality, base_width=self.base_width)
#         self.layer4 = self._make_layer(block, 512, layers[3], stride=2, cardinality=self.cardinality, base_width=self.base_width)
#         self.avgpool = nn.AvgPool2d(7, stride=1, padding=2)
#         self.fc = nn.Linear(2048 * block.expansion, num_classes)

#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
#                 m.weight.data.normal_(0, (2. / n)**.5)
#             elif isinstance(m, nn.BatchNorm2d):
#                 m.weight.data.fill_(1)
#                 m.bias.data.zero_()

# #     def _make_layer(self, block, planes, blocks, stride=1):
# #         layers = []
# #         layers.append(block(self.inplanes, planes, stride))
# #         self.inplanes = planes * block.expansion
# #         for i in range(1, blocks):
# #             layers.append(block(self.inplanes, planes))

# #         return nn.Sequential(*layers)

    
#     def _make_layer(self, block, planes, blocks, cardinality, base_width, stride=1):
#         layers = []
#         for i in range(1, blocks):
#             layers.append(block(self.inplanes, planes, stride, self.cardinality, self.base_width))
#             self.inplanes = planes * block.expansion
#         return nn.Sequential(*layers)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)

#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
#         logits = self.fc(x)
#         #probas = F.softmax(logits, dim=1)
#         return logits



# def resnet18(num_classes):
#     """Constructs a ResNet-18 model."""
#     model = ResNet(block=BottlenectBlock, 
#                    layers=[2, 2, 2, 2],
#                    num_classes=num_classes,
#                    grayscale=GRAYSCALE)
#     return model